In [1]:
cd ..

/Users/deepakduggirala/Documents/autonomous-robotics/bev-generation


In [8]:
import os
from pathlib import Path
import itertools
from io import BytesIO
import time
import json

import numpy as np
import scipy
import pandas as pd

import cv2

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from sklearn.neighbors import KNeighborsClassifier

from nuscenes.nuscenes import NuScenes

import lmdb

from tqdm import tqdm

import utils
from utils import bytes_to_array
import nusc_utils

from bev_gt import generate_gt_bev_map
from bev2 import BEV, generate_bev_seg_map

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
dataroot = Path('/Users/deepakduggirala/Documents/autonomous-robotics/v1.0-mini/').resolve()
nuscenes = NuScenes(version='v1.0-mini', dataroot=str(dataroot), verbose=False)
scene_idx = 0
sample_idx = 30

In [5]:
scene = nuscenes.scene[scene_idx]
sample = list(nusc_utils.sample_gen(nuscenes, scene))[sample_idx]

## Generate ground truths and write to lmdb file

In [6]:
main_db_path = dataroot / Path('lmdb/samples/GT_CAM_FRONT')
main_db = lmdb.open(path=str(main_db_path), map_size=int(50 * 1024 * 1024 * 1024))

In [41]:
# with main_db.begin(write=True) as write_txn:
#     for scene in nuscenes.scene:
#         for sample in tqdm(nusc_utils.sample_gen(nuscenes, scene)):
#             bev_gt_map = generate_gt_bev_map(nuscenes, scene, sample)
#             key = bytes(sample['data']['CAM_FRONT'], 'utf-8')
#             value = array_to_bytes(bev_gt_map)
#             write_txn.put(key, value)

39it [00:20,  1.89it/s]
40it [00:21,  1.88it/s]
41it [00:23,  1.76it/s]
41it [00:21,  1.87it/s]
41it [00:22,  1.84it/s]
40it [00:21,  1.86it/s]
41it [00:21,  1.88it/s]
41it [00:21,  1.87it/s]
40it [00:21,  1.89it/s]
40it [00:20,  1.91it/s]


## Generate interpolated BEV and compute IOU

In [9]:
results = []

with main_db.begin() as txn:
    for scene in nuscenes.scene:
        for sample in tqdm(nusc_utils.sample_gen(nuscenes, scene)):
            key = bytes(sample['data']['CAM_FRONT'], 'utf-8')
            value_bytes = txn.get(key)
            bev_gt_map = bytes_to_array(value_bytes)
            
            bev_seg_map, nusc_idx_to_color = generate_bev_seg_map(nuscenes, sample, 
                                                                  depth_intp_method='linear', 
                                                                  seg_cls_intp_method='nearest')
            pred, labels = utils.transform_pred_and_labels(bev_seg_map, bev_gt_map)
            iou_per_class, iou_per_sample = utils.compute_iou(pred, labels)
            
            results.append({
                'scene_token': scene['token'],
                'sample_token': sample['token'],
                'iou_per_class': list(iou_per_class),
                'iou_per_sample': iou_per_sample
            })

39it [00:15,  2.46it/s]
40it [00:16,  2.47it/s]
41it [00:17,  2.36it/s]
41it [00:16,  2.48it/s]
41it [00:17,  2.37it/s]
40it [00:16,  2.36it/s]
41it [00:17,  2.33it/s]
41it [00:17,  2.37it/s]
40it [00:17,  2.35it/s]
40it [00:17,  2.25it/s]


In [10]:
with open('iou-linear-linear-static-interpolation-v1.0-mini.json', 'w') as f:
    json.dump(results, f)

In [11]:
mean_iou_per_sample = np.array([r['iou_per_sample'] for r in results]).mean()
mean_iou_per_class = np.array([r['iou_per_class'] for r in results]).mean(axis=0)

mean_iou_per_sample, mean_iou_per_class

(0.18189610094152203,
 array([0.44281508, 0.0843298 , 0.11781198, 0.0472031 , 0.03979093,
        0.00216841, 0.02083152, 0.04235435, 0.01194333, 0.01138728,
        0.00088493, 0.00927088]))

In [12]:
labels = [x for i,x in enumerate(nusc_utils.NUSC_LIDAR_CLASS_NAMES) if i not in (1,3)]
df = pd.DataFrame(np.round(mean_iou_per_class.reshape(-1,12),3), columns=labels).T
df.to_csv('iou.csv', header=False)
df

,0
driveable_surface,0.443
sidewalk,0.084
car,0.118
truck,0.047
bus,0.040
trailer,0.002
construction_vehicle,0.021
pedestrian,0.042
motorcycle,0.012
bicycle,0.011
